In [1]:
import patsy as pa
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

from MakeMyPrior.elicitation_wrapper import expert_model
from MakeMyPrior.training import trainer
from MakeMyPrior.helper_functions import group_obs, Exponential_unconstrained, Normal_unconstrained
from MakeMyPrior.user_config import target_config, target_input
from MakeMyPrior.plot_helpers import plot_loss, plot_convergence, plot_priors

ModuleNotFoundError: No module named 'make_my_prior.target_quantities'

# Normal Model
## User specification

### General variables for the learning algorithm

In [ ]:
user_config = dict(                    
    B = 2**8,                          
    rep = 600,                         
    epochs = 6,#00,                      
    view_ep = 1,
    lr_decay = False,
    lr0 = 0.01, 
    lr_min = 0.01, 
    loss_dimensions = "m,n:B",          
    loss_scaling = "unscaled",         
    method = "hyperparameter_learning"  
    )
    
user_config

### Design matrix

(Only the first 10 observations are presented.)

In [ ]:
# design matrix
X =  pa.dmatrix("a*b", pa.balanced(a = 2, b = 3, repeat = 60))
dmatrix = tf.cast(X, dtype = tf.float32)
# contrast matrix
cmatrix = dmatrix[0:dmatrix.shape[1], :]

# show first 10 rows
dmatrix[:10, :]

## Model building

\begin{align}
    y_i &\sim \textrm{Normal}(\theta_i, s) \\
    \theta_i &= \beta_0 + \beta_1x_1 + \beta_2x_2 + \beta_3x_3 + \beta_4x_4 + \beta_5x_5\\
    \beta_k &\sim \textrm{Normal}(\mu_k, \sigma_k) \quad \textrm{ for }k=0,\ldots,5\\
     s &\sim \textrm{Exponential}(\nu)\\
\end{align}

### Prior distributions

Specify...

-   prior distribution family for model parameters
-   distributions for initializing hyperparameter values of parametric prior distributions
-   prior distribution of ideal expert with hyperparameter values representing the ground truth

In [ ]:
# true hyperparameter values for ideal_expert
true_mu = [0.12, 0.15, -0.02, -0.03, -0.02, -0.04]
true_sigma = [0.02, 0.02, 0.06, 0.06, 0.03, 0.03]
true_nu = 9.

# model parameters
parameters_dict = dict()
for i in range(6):
    parameters_dict[f"beta_{i}"] = {
            "family":  Normal_unconstrained(),
            "true": tfd.Normal(true_mu[i], true_sigma[i]),
            "initialization": [tfd.Normal(0.,0.1)]*2
            }
parameters_dict["sigma"] = {
        "family": Exponential_unconstrained(user_config["rep"]),
        "true": tfd.Exponential(true_nu),
        "initialization": [tfd.Normal(0.,0.1)]
        }

### Generative model

Necessary Input-Output structure for continuous likelihood:

-   **Input**: `parameters`
-   **Output**: `likelihood`, `ypred`, `epred`

In [ ]:
# generative model
class GenerativeModel(tf.Module):
    def __call__(self, 
                 parameters, # obligatory: samples from prior distributions; tf.Tensor
                 dmatrix,    # optional: design matrix; tf.Tensor
                 cmatrix,    # optional: contrast matrix; tf.Tensor
                 **kwargs    # obligatory: possibility for further keyword arguments is needed 
                 ):  
        
        # compute linear predictor term
        epred = parameters[:,:,0:6] @ tf.transpose(dmatrix)
        
        # define likelihood
        likelihood = tfd.Normal(
            loc = epred, 
            scale = tf.expand_dims(parameters[:,:,-1], -1))
        
        # sample prior predictive data
        ypred = likelihood.sample()
        
        # compute custom target quantity (here: group-differences)
        samples_grouped = group_obs(ypred, dmatrix, cmatrix)

        # compute mean difference between groups
        effect_list = []
        diffs = [(0,3), (1,4), (2,5)]
        for i in range(len(diffs)):
            # compute group difference
            diff = tf.math.subtract(
                samples_grouped[:, :, :, diffs[i][0]],
                samples_grouped[:, :, :, diffs[i][1]],
            )
            # average over individual obs within each group
            diff_mean = tf.reduce_mean(diff, axis=2)
            # collect all mean group differences
            effect_list.append(diff_mean)

        mean_effects = tf.stack(effect_list, axis=-1)
        
        return dict(likelihood = likelihood,     # obligatory: likelihood; callable
                    ypred = ypred,               # obligatory: prior predictive data
                    epred = epred,               # obligatory: samples from linear predictor
                    mean_effects = mean_effects  # optional: custom target quantity
                    )

## Loss components

(target quantities, elicitation techniques, combining losses)

In [ ]:
# define a custom function using the output from the generative model   
def custom_r2(ypred, epred, **kwargs):
    return tf.math.divide(tf.math.reduce_variance(epred, axis = -1), 
                          tf.math.reduce_variance(ypred, axis = -1))

# specify target quantity, elicitation technique and loss combination
t1 = target_config(target="R2", 
                   elicitation="histogram",
                   combine_loss="all",
                   custom_target_function = custom_r2)
t2 = target_config(target="group_means", 
                   elicitation="quantiles", 
                   combine_loss="by-group", 
                   quantiles_specs = (10, 20, 30, 40, 50, 60, 70, 80, 90))
t3 = target_config(target="mean_effects", 
                   elicitation="quantiles",
                   combine_loss="by-group",
                   quantiles_specs = (10, 20, 30, 40, 50, 60, 70, 80, 90))

target_info = target_input(t1, t2, t3)

target_info

### Number and Shape of loss components

In [ ]:
# ideal expert
expert_res_list, prior_pred_res = expert_model(1, user_config["rep"],
                               parameters_dict, GenerativeModel, target_info,
                               method = "ideal_expert",
                               dmatrix = dmatrix,
                               cmatrix = cmatrix,
                               dmatrix_fct = dmatrix)
                               
for i in range(len(expert_res_list)):
  print(f"loss component {i}, with shape = ", expert_res_list[i].shape)

## Training

In [ ]:
# simulation model and training
res_dict = trainer(expert_res_list, user_config["B"], user_config["rep"],
                   parameters_dict, user_config["method"], GenerativeModel,
                   target_info, user_config, loss_balancing = True,
                   dmatrix = dmatrix, cmatrix = cmatrix, dmatrix_fct = dmatrix)

## Results

In [ ]:
plot_loss(user_config, res_dict)

plot_convergence(user_config, res_dict)

plot_priors(res_dict, ideal_expert_dict)